# 优化过程

- 之前的文档介绍了一些基础的工具, 但并未介绍整个工作的流程
- 本篇文章介绍了整个神经网络搭建、训练、保存的工作流程
- 如果在未来的什么时候忘记了工作流程, 可以来这里查看

- - -

## 一、数据准备

在进行优化过程前, 我们需要准备好数据与网络, 我们将使用之前介绍的[Dataset与DataLoader](./2_Dataset_DataLoader_两个与数据加载相关的类.ipynb)进行数据加载, 并使用[torch.nn](./4_搭建神经网络的工具_torch.nn包.ipynb)搭建神经网络

需要的数据有：
1. 训练集与测试集-Dataset类的实例
2. 训练集与测试集的数据加载器-DataLoader类的实例
3. 神经网络模型-神经网络类的实例


准备数据集的代码如下：

In [1]:
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    transform=ToTensor(),
    download=True,
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    transform=ToTensor(),
    download=True
)

train_dataloader = DataLoader(dataset=training_data, batch_size=64)
test_dataloader = DataLoader(dataset=test_data, batch_size=64)


100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



定义神经网络的代码如下：

In [2]:

class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

创建一个神经网络的实例, 如果需要, 可以使用如cuda的硬件进行加速

In [3]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = NeuralNetwork().to(device=device)

Using cuda device


除此之外, 还需要设置一些超参数：
1. 学习率
2. bath_size
3. epoch

如下：

In [4]:
# 超参数的设置

learning_rate = 1e-3
batch_size = 64
# epochs=2意味着将在整个数据集上进行2次训练+测试的过程
epochs = 50

## 二、训练与测试过程的创立

- 在准备好数据后, 我们需要定义优化过程与测试过程
- 这两个过程往往以函数的形式出现
- 一般命名为`train_loop`与`test_loop`

In [5]:
# 定义train_loop函数
def train_loop(dataloader, model, loss_fn, optimizer):
    # 将模型设置为“训练模式”
    # 这个语句不是必须的, 但是加上能提升模型的性能, 并降低过拟合的概率
    model.train()

    # 获取数据集的大小, 用于打印当前训练的批次
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device=device), y.to(device=device)
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

上面的代码是`train_loop`函数的定义, 用于在给定的数据集上训练神经网络模型, 代码的含义如下：
1. `def train_loop(dataloader, model, loss_fn, optimizer):`：
   1. 这是一个名为 `train_loop` 的函数定义, 它接受四个参数：
      1. `dataloader`（数据加载器, 用于加载训练数据批次）
      2. `model`（神经网络模型）
      3. `loss_fn`（损失函数）
      4. `optimizer`（优化器）. 
2. `model.train()`：
   1. 将模型设置为训练模式. 这会启用训练模式特定的功能, 例如批次归一化和丢弃等. 
   2. 在PyTorch中, 调用 `model.train()` 方法并不是必须的, 但通常建议在训练阶段显式调用它. 
   3. 调用 `model.train()` 主要是为了启用一些训练模式下特有的功能和行为, 以确保模型在训练过程中正确执行, 并且适应于训练数据的统计特性. 
   4. 具体来说, `model.train()` 方法会做以下几件事情：
      1. 激活训练模式：启用一些训练模式下特有的操作, 例如批次归一化层使用当前批次的统计信息来标准化输入, 丢弃层执行丢弃操作等. 
      2. 启用梯度跟踪：默认情况下, PyTorch会在训练模式下启用梯度跟踪, 即会追踪模型参数的梯度信息, 以便后续执行反向传播和参数更新操作. 
   5. 虽然调用 `model.train()` 方法不是强制性的, 但在训练过程中使用它可以确保模型处于正确的模式下, 并且可以获得预期的行为. 
   6. 另外, 即使在一些情况下, 如评估模型或进行推理时, 调用 `model.eval()` 来将模型设置为评估模式也是一个好的实践. 
3. `size = len(dataloader.dataset)`：
   1. 计算训练数据集的总样本数, 以便在训练过程中跟踪进度. 
4. `for batch, (X, y) in enumerate(dataloader):`：
   1. 这是一个迭代训练数据加载器的循环. 
   2. dataloader是一个可迭代对象, 加上`enumerate`之后, 就变成了一个[可索引对象](../../../Python_Learn/python语法基础/2_数组类元素_列表、元组、字典、迭代器/可索引.ipynb), 此处使用可索引`enumerate`而非可迭代`iteror`的原因在于, 我们需要获取当前处理的是第几个数据.
      1. 在使用 `enumerate` 而不是 `iteror` 的情况下, 主要是为了获取迭代对象的索引. 虽然 `iteror` 可以用于迭代对象, 但它并不提供索引信息. 
      2. 具体来说, 使用 `enumerate` 的好处是可以**同时获取索引**和**对应的元素**, 这在某些情况下非常有用, 特别是在需要对数据进行索引处理或者跟踪处理进度时. 
         1. 比如在训练神经网络时, 经常需要使用 `enumerate` 来遍历数据集, 并获取每个批次的索引, 以便记录训练进度或者进行其他操作. 
      3. 另外, 使用 `enumerate` 还可以提高代码的可读性和简洁性, 因为它提供了一种直观的方式来同时获取索引和元素, 而不需要额外的变量来记录索引. 这使得代码更加简洁和易于理解. 
      4. 总之, 虽然 `iteror` 也可以用于迭代对象, 但如果需要获取索引信息或者希望代码更加简洁和可读, 那么使用 `enumerate` 通常是更好的选择. 
   3. 在循环中, `enumerate(dataloader)`每次返回两个数据
      1. 第一个数据是当前读取数据处于整个可索引对象中的位置, 即下标(索引)信息
      2. 第二个数据是一个包含输入数据和标签的元组 (X, y)，其中 X 是输入数据，y 是相应的标签. 
   4. 因此, 循环中的每一次操作, 我们都将获得一个批次的输入数据 `X` 和相应的标签 `y`, 并用`batch`获得他们当前是第几个数据

5. `pred = model(X)`：
   1. 使用模型进行前向传播, 得到模型的预测结果 `pred`. 
6. `loss = loss_fn(pred, y)`：
   1. 使用给定的损失函数 `loss_fn`, 计算预测结果与真实标签之间的损失. 
7. `loss.backward()`：
   1. 计算梯度：反向传播损失, 计算损失相对于模型参数的梯度. 
8. `optimizer.step()`：
   1. 使用优化器来更新模型参数, 根据反向传播计算得到的梯度. 
9.  `optimizer.zero_grad()`：
    1.  清零优化器中所有参数的梯度, 以便下一次迭代计算新的梯度. 
10. `if batch % 100 == 0:`：
    1.  每隔一定的批次数（这里是每100个批次）, 执行以下操作. 
11. `loss, current = loss.item(), batch * len(X) + len(X)`：
    1.  获取当前批次的损失值, 并计算当前处理的样本数量. 
12. `print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")`：
    1.  打印当前批次的损失值以及当前处理的样本数量. 
    2.  在这个语句中，`{loss:>7f}` 和 `{current:>5d}` 分别是格式化字符串的一部分，它们用于对打印的变量进行格式化，以确保输出的字符串符合指定的格式。下面解释每个部分的含义：
        1. `{loss:>7f}`:
           - `loss` 是一个浮点数变量，表示损失值。
           - `:` 后的 `>` 是格式化指令，表示右对齐。如果指定 `<` 则为左对齐。
           - `7` 表示最小字段宽度为7个字符，如果输出的值不够7个字符宽度，则会在左侧填充空格以达到7个字符宽度。
           - `f` 表示格式化为浮点数。
           - 综合起来，`{loss:>7f}` 的含义是将损失值格式化为浮点数，保留小数点后的6位数字，并且右对齐，总宽度为7个字符。

        2. `{current:>5d}`:
           - `current` 是一个整数变量，表示当前处理的样本数量。
           - `:` 后的 `>` 是格式化指令，表示右对齐。如果指定 `<` 则为左对齐。
           - `5` 表示最小字段宽度为5个字符，如果输出的值不够5个字符宽度，则会在左侧填充空格以达到5个字符宽度。
           - `d` 表示格式化为整数。
           - 综合起来，`{current:>5d}` 的含义是将当前处理的样本数量格式化为整数，并且右对齐，总宽度为5个字符。
      - 这些格式化指令可以用于控制输出字符串的格式，使其对齐和易读。

In [6]:
# 定义test_loop函数
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X,y = X.to(device=device), y.to(device=device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

这段代码定义了一个名为 `test_loop` 的函数, 用于在给定的数据加载器上评估模型的性能. 下面逐行解释这段代码：

1. `model.eval()`：
   1. 将模型设置为评估模式. 在评估模式下, 模型会禁用一些训练模式下特有的功能, 例如批次归一化和丢弃层的操作. 
   2. 虽然不是必需的, 但是还是推荐在test阶段进行该操作, 因为它确保了模型处于正确的模式下, 能够取得更好的效果. 
2. `size = len(dataloader.dataset)` 和 `num_batches = len(dataloader)`：
   1. 分别计算数据加载器中数据集的总大小和数据批次的数量. 
3. `test_loss, correct = 0, 0`：
   1. 初始化测试损失和正确预测的样本数量. 
4. `with torch.no_grad():`：
   1. 使用 `torch.no_grad()` 上下文管理器, 确保在评估模式下不会计算梯度. 这有助于减少不必要的梯度计算和内存使用, 特别是对于 `requires_grad=True` 的张量来说. 
5. `for X, y in dataloader:`：
   1. 遍历数据加载器中的每个数据批次. 在每次迭代中, `X` 是输入数据, `y` 是相应的标签. 
6. `pred = model(X)`：
   1. 使用模型进行前向传播, 得到预测结果 `pred`. 
7. `test_loss += loss_fn(pred, y).item()`：
   1. 计算并累加当前数据批次的预测损失. 
   2. 这里使用了损失函数 `loss_fn` 对模型的预测结果 `pred` 和真实标签 `y` 进行计算. 
8. `correct += (pred.argmax(1) == y).type(torch.float).sum().item()`：
   1. 计算并累加当前数据批次中模型正确预测的样本数量. 
   2. 首先, `pred.argmax(1)` 返回每个样本的预测类别, 
   3. 然后将其与真实标签 `y` 进行比较, 得到一个布尔值的张量, 
   4. 最后将布尔值的张量转换为浮点型张量, 并求和. 
9.  `test_loss /= num_batches` 和 `correct /= size`：
    1.  计算平均测试损失和正确预测的样本比例. 
10. `print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")`：
    1.  打印评估结果, 包括准确率和平均损失. 
    2.  使用格式化字符串 `{(100*correct):>0.1f}` 将准确率格式化为百分比, 保留一位小数；
    3.  `{test_loss:>8f}` 将平均损失格式化为浮点数, 总宽度为8个字符. 

## 三、优化过程的创建

现在, 我们将初始化「损失函数」与「优化器」, 并将这两个对象传递给`train_loop`与`test_loop`.

如果有意愿, 可以随意调整`epochs`的值

初始化「损失函数」与「优化器」的代码如下：

In [7]:
# 初始化「损失函数」与「优化器」

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr = learning_rate)

## 四、训练过程

- 一般来说, 在循环中重复调用`train_loop`与`test_loop`即可完成训练过程
- 该例子中没有打印损失函数的值, 如果需要可以记录并保存

In [8]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    # 每个epch中, 应先有一个训练过程
    train_loop(
        dataloader=train_dataloader,
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer
    )

    # 每个epch中, 在train后, 应该有一个test过程
    test_loop(
        dataloader=test_dataloader,
        model=model,
        loss_fn=loss_fn
    )

    
print("Done!")

Epoch 1
-------------------------------
loss:2.292420 [   64/60000]
loss:2.276186 [ 6464/60000]
loss:2.254463 [12864/60000]
loss:2.255749 [19264/60000]
loss:2.229204 [25664/60000]
loss:2.199419 [32064/60000]
loss:2.212619 [38464/60000]
loss:2.176709 [44864/60000]
loss:2.170320 [51264/60000]
loss:2.141760 [57664/60000]
Test Error: 
 Accuracy: 47.2%, Avg loss: 2.132148 

Epoch 2
-------------------------------
loss:2.142294 [   64/60000]
loss:2.128763 [ 6464/60000]
loss:2.060271 [12864/60000]
loss:2.089245 [19264/60000]
loss:2.018003 [25664/60000]
loss:1.953646 [32064/60000]
loss:1.983658 [38464/60000]
loss:1.901944 [44864/60000]
loss:1.916574 [51264/60000]
loss:1.830000 [57664/60000]
Test Error: 
 Accuracy: 57.1%, Avg loss: 1.834447 

Epoch 3
-------------------------------
loss:1.871033 [   64/60000]
loss:1.839510 [ 6464/60000]
loss:1.710800 [12864/60000]
loss:1.768341 [19264/60000]
loss:1.637314 [25664/60000]
loss:1.595488 [32064/60000]
loss:1.613824 [38464/60000]
loss:1.527880 [44864

## 五、网络结构与参数的保存

在训练完毕后, 我们可以将网络结构与训练获得的参数保存下来, 防止训练数据的丢失, 工作流程如下：
1. 保存网络结构
2. 保存网络参数

In [15]:
# 网络结构保存
torch.save(model, './data/model_and_weights_for_z/model.pth')

# 网络状态与参数保存
torch.save(model.state_dict(), './data/model_and_weights_for_z/model_state_dict.pth')

# 网络的使用

## 六、网络结构与参数的加载

如果保存了网络结构与参数, 则可以在其他程序中使用该网络, 加载过程如下：

In [16]:
# 加载网络
model = torch.load('./data/model_and_weights_for_z/model.pth')

# 加载网络参数
model_state_dict = torch.load('./data/model_and_weights_for_z/model_state_dict.pth')

# 将网络参数加载入网络中
model.load_state_dict(state_dict=model_state_dict)

<All keys matched successfully>

## 七、网络的使用

In [55]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

# 从数据集中取标签与数据
x, y = test_data[0][0], test_data[0][1]

# 使用no_grad上下文管理器
with torch.no_grad():
    x = x.to(device)
    
    # 直接使用model(图像)即可调用网络
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
